In [178]:
import warnings
warnings.filterwarnings("ignore")
import os

os.sys.path.insert(0, '../script')

from config import *
from webnlg import *

In [179]:
my_models_paths = !ls -d -1 ../data/models/*.txt

my_models_paths

['../data/models/5 - Model - Template Based - roots_3ae48a5d03a715d609dc8d73f82ac5682cba128d.txt',
 '../data/models/5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2.txt',
 '../data/models/5 - Model - Template Based - roots_55f9d82b420c5cbc6a01d5a65082414b87a81195.txt']

In [180]:
models_paths = my_models_paths + SUBMISSIONS_FILEPATHS
models_paths

['../data/models/5 - Model - Template Based - roots_3ae48a5d03a715d609dc8d73f82ac5682cba128d.txt',
 '../data/models/5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2.txt',
 '../data/models/5 - Model - Template Based - roots_55f9d82b420c5cbc6a01d5a65082414b87a81195.txt',
 '../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt',
 '../data/webnlg2017/submissions/melbourne/final_result.txt',
 '../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt',
 '../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/smt_test.out.ordered',
 '../data/webnlg2017/submissions/tilburg/template_test.out.ordered',
 '../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt',
 '../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt',
 '../data/webnlg2017/submissions/baseline_sorted.txt']

In [181]:
model_results = {}

for model_path in models_paths:
    
    with open(model_path) as f:
        
        model_results[model_path] = [line[:-1] for line in f]

In [182]:
import pandas as pd
pd.set_option('max_colwidth', 1000)
from nltk.translate.bleu_score import sentence_bleu

lex = WebNLGCorpus.load("test_with_lex")

s_multiwordexpressions = lex.mdf.m_subject.str.lower().str.replace('_', ' ').str.split().tolist()
o_multiwordexpressions = lex.mdf.m_object.str.lower().str.replace('_', ' ').str.split().tolist()

all_multiwordexpressions = s_multiwordexpressions + o_multiwordexpressions
all_multiwordexpressions = list(set([tuple(s) for s in all_multiwordexpressions]))

from nltk.tokenize.mwe import MWETokenizer

t = MWETokenizer(all_multiwordexpressions)

from textacy.preprocess import preprocess_text

def preprocess(ref):
    
    ref = preprocess_text(ref, no_accents=True, lowercase=True, no_punct=True)
    
    return t.tokenize(ref.split())


def print_entry_and_models(e):
    
    print(e)
    print('\n')
    
    idx = int(e.idx().split('_')[1])
    
    references = [preprocess(ref) for ref in e.ldf.ltext.tolist()]
    
    texts = []
    bleus = []
    
    for model_path in models_paths:
        
        model_text = model_results[model_path][idx]
        
        bleu = sentence_bleu(references, preprocess(model_text))
        
        texts.append(model_text)
        bleus.append(bleu)
        
    models_names = [model_path.split('/')[-1] for model_path in models_paths]
        
    return pd.DataFrame({'models': models_names,
                         'texts': texts,
                         'bleu': bleus}).sort_values('bleu', ascending=False)

In [183]:
test_with_lex = WebNLGCorpus.load('test_with_lex')

In [184]:
entries = list(test_with_lex)

In [185]:
i = 0

In [189]:
e = entries[i]
i += 1

df = print_entry_and_models(e)

my_model_bleu = df[df.models == '5 - Model - Template Based - roots_3ae48a5d03a715d609dc8d73f82ac5682cba128d.txt'].bleu.values[0]

if df.bleu.max() - my_model_bleu > .5:
    
    print('foi')

Triple info: {'category': 'Airport', 'eid': 'Id4', 'idx': '0_3', 'ntriples': 1}

	Modified triples:

Afonso_Pena_International_Airport | ICAO_Location_Identifier | "SBCT"


	Lexicalizations:

Afonso Pena International Airport ICAO Location Idenitifier is SBCT.
SBCT is the ICAO location identifier of Afonso Pena International Airport.


foi


In [190]:
df

,bleu,models,texts
9,1.000000e+00,Submission-UIT-DANGNT-CLNLP.txt,"sbct is the icao location identifier of afonso pena international airport , ."
1,7.652059e-01,5 - Model - Template Based - roots_4e9245cbbc05e4fdf0de404fbada20d8294127b2.txt,The ICAO Location Identifier of Afonso Pena International Airport is SBCT.
4,7.652059e-01,final_result.txt,the icao location identifier of afonso pena international airport is sbct .
5,7.652059e-01,PKUWriter_results.txt,the icao location identifier of afonso pena international airport is sbct .
11,7.652059e-01,baseline_sorted.txt,the icao location identifier of afonso pena international airport is sbct .
10,5.307712e-01,UPF_All_sent_final.txt,the icao location identifier of the afonso pena international airport is sbct .
2,9.712949e-78,5 - Model - Template Based - roots_55f9d82b420c5cbc6a01d5a65082414b87a81195.txt,Afonso Pena International Airport ICAO Location Identifier is SBCT.
3,6.608447e-78,ADAPTcentreWebNLGsubmission.txt,afonso pena international airport 's icao location identifier is sbct .
7,6.206022e-78,smt_test.out.ordered,sbct airport 's icao location identifier .
6,1.090574e-154,nmt_test.out.ordered,the location identifier for afonso pena international airport is sbct .


In [191]:
e.get_data()

[{'m_subject': 'Afonso_Pena_International_Airport',
  'm_predicate': 'ICAO_Location_Identifier',
  'm_object': '"SBCT"'}]